In [1]:
import os
import sys
import json
import mlflow
import pandas as pd
import torch
import torch.nn as nn
import joblib
import matplotlib.pyplot as plt
from pathlib import Path
from tempfile import TemporaryDirectory
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import tqdm

sys.path.append('..')
from src.mlflow_utils import configure_mlflow, get_data, find_latest_run_id_by_experiment_and_stage, load_config

In [2]:
CONFIG = load_config()

In [3]:
def log_data_characteristics(data: pd.DataFrame, prefix: str) -> None:
    """Log comprehensive data characteristics"""
    mlflow.log_metrics({
        f"{prefix}_samples": data.shape[0],
        f"{prefix}_features": data.shape[1],
        f"{prefix}_nan_count": data.isna().sum().sum()
    })

In [4]:
class AutoencoderMLflowWrapper(mlflow.pyfunc.PythonModel):
    """MLflow-compatible wrapper for Autoencoder model"""
    def __init__(self, autoencoder, encoder):
        super().__init__()
        self.autoencoder = autoencoder
        self.encoder = encoder
        
    def predict(self, context, model_input):
        with torch.no_grad():
            tensor_input = torch.FloatTensor(model_input.values)
            return self.encoder(tensor_input).numpy()

/home/nick/snap/code/181/.local/share/virtualenvs/fraud-detection-mlops-5wXwr5xg/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:134: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


In [5]:
def create_autoencoder(config: dict, input_dim: int) -> tuple[nn.Module, torch.device]:
    """Autoencoder training pipeline with MLflow integration"""
    # Model architecture
    params = config["models"]["autoencoder"]["params"]
    
    class Autoencoder(nn.Module):
        def __init__(self, input_dim):
            super().__init__()
            self.enc1 = nn.Linear(input_dim ,params["hidden_dims"][0])
            self.enc_act1 = nn.ReLU()
            self.enc2 = nn.Linear(params["hidden_dims"][0], params["hidden_dims"][1])
            self.enc_bn = nn.BatchNorm1d(params["hidden_dims"][1])
            self.enc_act2 = nn.ReLU()
            self.enc3 = nn.Linear(params["hidden_dims"][1], params["encoding_dim"])
            self.dropout = nn.Dropout(params["dropout"])
            self.dec1 = nn.Linear(params["encoding_dim"], params["hidden_dims"][1])
            self.dec_act1 = nn.ReLU()
            self.dec2 = nn.Linear(params["hidden_dims"][1], params["hidden_dims"][0])
            self.dec_act2 = nn.ReLU()
            self.dec3 = nn.Linear(params["hidden_dims"][0], input_dim)

        def forward(self, x):
            enc1_out = self.enc_act1(self.enc1(x))
            enc2_out = self.enc_act2(self.enc_bn(self.enc2(enc1_out)))
            encoding = self.enc3(enc2_out)
            drop = self.dropout(encoding)
            dec1_out = self.dec_act1(self.dec1(drop))
            dec2_in = dec1_out + enc2_out
            dec2_out = self.dec_act2(self.dec2(dec2_in))
            dec3_in = dec2_out + enc1_out
            output = self.dec3(dec3_in)
            return output
        
    model = Autoencoder(input_dim)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # Log model architecture
    mlflow.log_params({
        "input_dim": input_dim,
        **params,
        "optimizer": "Adam",
        "loss_fn": "SmoothL1Loss"
    })
    
    return model, device

In [6]:
# Define Encoder
class Encoder(nn.Module):
    def __init__(self, autoencoder):
        super(Encoder, self).__init__()
        self.enc1 = autoencoder.enc1
        self.enc_act1 = autoencoder.enc_act1
        self.enc2 = autoencoder.enc2
        self.enc_bn = autoencoder.enc_bn
        self.enc_act2 = autoencoder.enc_act2
        self.enc3 = autoencoder.enc3

    def forward(self, x):
        x = self.enc_act1(self.enc1(x))
        x = self.enc_act2(self.enc_bn(self.enc2(x)))
        x = self.enc3(x)
        return x

In [7]:
def filter_non_fraud(features, target):
    non_fraud_mask = (target == 0).values.flatten()
    non_fraud_features = features[non_fraud_mask]
    return non_fraud_features

In [8]:
def train(epochs, autoencoder, train_loader, val_loader, optimizer, criterion, device, save_dir, config):
    best_loss = float('inf')
    patience_counter = 0
    train_losses = []
    val_losses = []
    
    epoch_bar = tqdm(range(epochs), desc='Training Autoencoder', unit='epoch')
    for epoch in epoch_bar:
        # Training phase
        autoencoder.train()
        epoch_train_loss = 0
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}', leave=False, unit='batch'):
            batch = batch[0].to(device)
            optimizer.zero_grad()
            outputs = autoencoder(batch)
            loss = criterion(outputs, batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(autoencoder.parameters(), 1.0)
            optimizer.step()
            epoch_train_loss += loss.item() * batch.size(0)

        # Validation phase
        autoencoder.eval()
        epoch_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch[0].to(device)
                outputs = autoencoder(batch)
                loss = criterion(outputs, batch)
                epoch_val_loss += loss.item() * batch.size(0)

        # Calculate metrics
        train_loss = epoch_train_loss / len(train_loader.dataset)
        val_loss = epoch_val_loss / len(val_loader.dataset)
        train_losses.append(train_loss)
        val_losses.append(val_loss)

        # Update progress bar
        epoch_bar.set_postfix({
            'Train Loss': f'{train_loss:.4f}',
            'Val Loss': f'{val_loss:.4f}',
            'LR': f'{optimizer.param_groups[0]["lr"]:.2e}'
        })

        mlflow.log_metrics({
            "train_loss": train_loss,
            "val_loss": val_loss,
            "learning_rate": optimizer.param_groups[0]['lr']
        }, step=epoch)

        patience = config["models"]["autoencoder"]["training_params"]["patience"]

        # Early stopping and model checkpointing
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
            torch.save(autoencoder.state_dict(), Path(save_dir) / "best_model.pth")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break

    return train_losses, val_losses

In [9]:
def plot_history(train_losses, val_losses, figsize=(10, 5)):
    plt.figure(figsize=figsize)
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Autoencoder Training Progress')
    plt.xlabel('Epochs')
    plt.ylabel('MSE Loss')
    plt.legend()
    plt.grid(True)

    with TemporaryDirectory() as tmp_dir:
        plot_path = os.path.join(tmp_dir, "training_history.png")
        plt.savefig(plot_path)
        mlflow.log_artifact(plot_path)
        
    plt.close()

In [10]:
def feature_engineering_pipeline(config: dict): 
    # Log environment details
    mlflow.log_params({
        "pytorch_version": torch.__version__,
        "cuda_available": torch.cuda.is_available(),
        "device_count": torch.cuda.device_count() if torch.cuda.is_available() else 0
    })
    # Link to preprocessing run
    preprocessing_run_id = find_latest_run_id_by_experiment_and_stage(
        config["experiment_names"]["preprocessing"],
        config["run_names"]["preprocessing"]
    )
    if not preprocessing_run_id:
        raise ValueError("No successful preprocessing run found")
        
    mlflow.log_param("preprocessing_run_id", preprocessing_run_id)
    mlflow.set_tags({
        "stage": "feature_engineering",
        "dataset_type": "tabular",
        "task": "reconstruction",
        "parent_run": preprocessing_run_id
    })

    # Retrieve processed data from MLflow
    processed_data = get_data(preprocessing_run_id, config["dataset"], config["artifacts"]["data"]["processed"])
    
    # Log data characteristics
    mlflow.log_metrics({
        "train_samples_retrieved": processed_data["X_train"].shape[0],
        "val_samples_retrieved": processed_data["X_val"].shape[0],
        "test_samples_retrieved": processed_data["X_test"].shape[0]
    })

    # Filter non-fraud cases
    X_train_non_fraud = filter_non_fraud(processed_data["X_train"], processed_data["y_train"])
    X_val_non_fraud = filter_non_fraud(processed_data["X_val"], processed_data["y_val"])

    # Create datasets
    train_dataset = TensorDataset(torch.tensor(X_train_non_fraud.values, dtype=torch.float32))
    val_dataset = TensorDataset(torch.tensor(X_val_non_fraud.values, dtype=torch.float32))

    batch_size = config["models"]["autoencoder"]["training_params"]["batch_size"]
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count())
    val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=os.cpu_count())

    input_dim = X_train_non_fraud.shape[1]
    training_params = config["models"]["autoencoder"]["training_params"]

     # Initialize model and training components
    autoencoder, device = create_autoencoder(config, input_dim)
    criterion = nn.SmoothL1Loss()
    optimizer = torch.optim.Adam(
        autoencoder.parameters(),
        lr=training_params["lr"],
        weight_decay=training_params["weight_decay"]
    )
    
    # Log training configuration
    mlflow.log_params(training_params)
    mlflow.log_dict(config, "training_config.json")

    epochs = training_params["epochs"]
    
    with TemporaryDirectory() as save_dir:
        train_losses, val_losses = train(epochs, autoencoder, train_loader, val_loader, optimizer, criterion, device, save_dir, config)
        plot_history(train_losses, val_losses)
        autoencoder.load_state_dict(torch.load(Path(save_dir) / "best_model.pth"))
        
    # Generate and log encoded features
    autoencoder.eval()
    encoder = Encoder(autoencoder).eval()

    mlflow.pytorch.log_model(
        pytorch_model=encoder,
        artifact_path="encoder",
        registered_model_name="FraudEncoder"
    )

    split_names = config["dataset"]["split_names"]
    split_dirs = config["dataset"]["split_dirs"]
    engineered_features_base_path = config["artifacts"]["data"]["engineered"]
    
    with torch.no_grad():
        with TemporaryDirectory() as tmp_dir:
            for split_name, split_dir in zip(split_names, split_dirs):
                # TODO: Concat with original features
                split_name = f"X_{split_name}"
                tensor_data = torch.FloatTensor(processed_data[split_name].values).to(device)
                encoded = encoder(tensor_data).cpu().numpy()
                pd.DataFrame(encoded).to_parquet(Path(tmp_dir) / f"{split_name}.parquet")
                mlflow.log_artifact(Path(tmp_dir) / f"{split_name}.parquet", f"{engineered_features_base_path}/{split_dir}")

In [11]:
if __name__ == "__main__":
    experiment_name = CONFIG["experiment_names"]["feature_engineering"]
    configure_mlflow(experiment_name)
    with mlflow.start_run(run_name=CONFIG["run_names"]["feature_engineering"]) as run:
        feature_engineering_pipeline(CONFIG)
        print("Feature engineering pipeline completed. Run ID:", mlflow.active_run().info.run_id)

Training Autoencoder:   0%|          | 0/100 [00:00<?, ?epoch/s]

Epoch 1:   0%|          | 0/756 [00:00<?, ?batch/s]

Epoch 2:   0%|          | 0/756 [00:00<?, ?batch/s]

Epoch 3:   0%|          | 0/756 [00:00<?, ?batch/s]

Epoch 4:   0%|          | 0/756 [00:00<?, ?batch/s]

...

Epoch 98:   0%|          | 0/756 [00:00<?, ?batch/s]

Epoch 99:   0%|          | 0/756 [00:00<?, ?batch/s]

Epoch 100:   0%|          | 0/756 [00:00<?, ?batch/s]

2025/02/06 18:44:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'FraudEncoder' already exists. Creating a new version of this model...
Created version '17' of model 'FraudEncoder'.


Feature engineering pipeline completed. Run ID: 42c5dd290a074514afed5bf25f632839
